# StyleGAN2-ADA-PyTorch

## Please Read
This StyleGAN2-ADA-PyTorch repository (including this Colab notebook) was forked from [Derrick Schultz](https://github.com/dvschultz/stylegan2-ada-pytorch), which was forked from [Nvidia's original repo](https://github.com/NVlabs/stylegan2-ada-pytorch). A huge thank you to Derrick for developing these tools to make GANs accessible for artists and newcomers, and for generously sharing his knowledge and resources **for free**. Without artists/educators like this, we would be spending much more time (and money) getting these things working. If you find that you're using this Colab notebook extensively, and especially if you're using it to make work to display outside the classroom, **_please be sure to credit [Derrick Schultz](https://www.instagram.com/dvsmethid/?hl=en)_**, and I encourage you all to consider signing up for his [Patreon](https://www.patreon.com/bustbright) or [YouTube channel](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA/join). You can also send him a one-time payment on [Venmo](https://venmo.com/Derrick-Schultz). Derrick also has a Slack channel dedicated to this stuff, which you can access as a Patreon subscriber.

~ ~ ~ ~ ~ ~ ~ ~ ~

This Notebook was last updated: March 30, 2021 by [Doug Rosman](https://dougrosman.com/).

Additional Colab and Python help from Blake Fall-Conroy.

~ ~ ~ ~ ~ ~ ~ ~ ~

## Connecting to a GPU instance

Let’s start by checking to see what GPU we’ve been assigned. This will also connect the notebook to a GPU runtime, meaning you'll now officially start using your ~8 hours of free GPU time.

Ideally the GPU we get is a V100, but a P100 is fine too. Other GPUs may lead to issues.

In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-5256f904-909a-ad5f-c4f6-3a1bf2b0983e)


# Setup
First, let’s connect our Google Drive account. This is optional but highly recommended. (You should definitely do this).

After completing this step, I recommend opening another tab in your browser to your Google Drive folder. Even though you can access your Google Drive from this notebook, it's much easier to work with your files in the regular Google Drive interface.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install repo/Move into StyleGAN2-ADA-PyTorch repo if already installed

The next cell will install the StyleGAN repository in Google Drive. If you have already installed it it will just move into that folder. If you don’t have Google Drive connected it will just install the necessary code in Colab.

In [3]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dougrosman/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dougrosman/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex

/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch
     |████████████████████████████████| 112kB 5.3MB/s 


### **_Do not run this cell unless explicitly told to by Doug_**

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "dougrosman"
!git config --global user.email "drosman@saic.edu"
!git fetch origin
!git pull
# !git checkout origin/main -- train.py generate.py

/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/dougrosman/stylegan2-ada-pytorch
   3330aa3..beca8f2  main       -> origin/main
error: You have not concluded your merge (MERGE_HEAD exists).
hint: Please, commit your changes before merging.
fatal: Exiting because of unfinished merge.


# Dataset Preparation - Uploading your data set

Upload a .zip of square images to the `datasets` folder. Previously you had to convert your model to .tfrecords. That’s no longer needed :)

This snippet uses an augmented version of the original dataset_tool.py from Nvidia's repo that can handle a wider variety of image types without failing.

### **Mac Users**

The .zip folders you make on you Mac are likely to _fail_ if you use the standard method of compressing your folders (i.e. right-clicking on the folder and selecting _Compress "folder"_). Your Mac automatically creates a hidden folder in the .zip that becomes visible when you extract the .zip files on a non-Mac (like in this Colab, for instance). This causes problems.

![bad zip](https://raw.githubusercontent.com/dougrosman/stylegan2-ada-pytorch/main/images/bad_zip_apple.png)

To solve this, download and install a third-party app called [Keka](https://www.keka.io/en/), and use that to create a .zip folder.
#### **To compress a folder with Keka:**
1. Open Keka
2. Check the box next to '**Exclude Mac resource forks**'
3. Select '**ZIP**' from the top-right dropdown.
4. Drag and drop your folder directly onto Keka
5. It may ask you where to save your file. Select your destination and compress your folder. That .zip is safe to be uploaded to this Colab.

![good zip with Keka](https://raw.githubusercontent.com/dougrosman/stylegan2-ada-pytorch/main/images/keka.png)

# Batch cropping and resizing your images

If your the images in your .zip folder are not uniformly square, run this cell to automatically resize and center crop your images to the specified resolution. Set the following arguments in the cell below (you'll need to scroll all the way to the right to change the width/height). After setting them, run the cell. This will take a couple minutes, depending on the quantity of images in your data set. **If this process fails for any reason, please let Doug know.**

* __`--width="1024"`__ - set this to 1024 (or 512, or 256, depending on your needs)
* __`--height="1024"`__ - set this to the same value as your --width
* __`--source=/path/to/your/dataset.zip`__ (just change the dataset name at the end of the path to the name of the data set you want to process)
* __`--dest=/path/to/your/dataset-processed.zip`__ (same as above, but add "-processed" to the end of the filename (e.g. mydata-processed.zip)

In [ ]:
!python dataset_tool-jpg.py --source=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/datasets/banana.zip --dest=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/datasets/banana-processed.zip --width=1024 --height=1024 --transform="center-crop"

100% 98/98 [00:16<00:00,  5.79it/s]


## Train model

Below are a series of variables you need to set to run the training. You probably won’t need to touch most of them.

* `dataset_path`: this is the path to your .zip file (make sure that if you ran the data processing step above, you point to the processed data set.)
* `resume_from`: if you’re starting a new dataset I recommend `'ffhq1024'` or `'./pretrained/wikiart.pkl'` (You can download other pretrained models [here](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/). (for 512x512, you can use afhqcat.pkl, afhqdog.pkl,  afhqwild.pkl, or brecahad.pkl)
* `mirror_x` and `mirror_y`: Allow the dataset to use horizontal or vertical mirroring.
* `snapshot_count`: this determines how frequently your progress will be saved. If set to '1', a .pkl file will be saved with each tick (every ~30 mins or so). **I recommend this.** This will end up using a lot of space in your Google Drive, since you'll be saving a lot pickles, but if you have enough space, it's worth it. Setting this to '2' is also reasonable. This way, you are less likely to lose hours of training progress when Google shuts down your training.

In [ ]:
#required: definitely edit these!
dataset_path = './datasets/keyboard_hands-1024.zip'
resume_from = './pretrained/wikiart.pkl' # a good pretrained model for 1024x1024 images
# resume_from = '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00000-apple-processed-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000020.pkl'
aug_strength = 0
train_count = 0
mirror_x = True

#broken, don't use for now :(
#mirror_y = False

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bg'
config = '11gb-gpu'
snapshot_count = 1

In [ ]:
!python train.py --gpus=1 --cfg=$config --metrics=None --outdir=./results --data=$dataset_path --snap=$snapshot_count --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=False --nkimg=$train_count

### Resume Training

Once Colab has shutdown, you’ll need to resume your training. Reset the variables above, particularly the `resume_from` and `aug_strength` settings.

1. Point `resume_from` to the last .pkl you trained (you’ll find these in the `results` folder)
2. Update `aug_strength` to match the augment value of the last pkl file. Often you’ll see this in the console, but you may need to look at the `log.txt`. Updating this makes sure training stays as stable as possible.
3. You may want to update `train_count` to keep track of your training progress.

Once all of this has been reset, run that variable cell and the training command cell after it.

## Convert Legacy Model

If you have an older version of a model (Tensorflow based StyleGAN, or Runway downloaded .pkl file) you’ll need to convert to the newest version. If you’ve trained in this notebook you do **not** need to use this cell.

`--source`: path to model that you want to convert

`--dest`: path and file name to convert to.

In [ ]:
!python legacy.py --source=/content/drive/MyDrive/runway.pkl --dest=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/runway.pkl

## Testing/Inference

Also known as "Inference", "Evaluation" or "Testing" the model. This is the process of usinng your trained model to generate new material, usually images or videos.

### Generate Single Images

`--network`: Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`: This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation`: Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 


In [ ]:
!python generate.py --outdir=./generated/kbhands138-random-0-49 --trunc=1.0 --seeds=0-49 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl

### Truncation Traversal

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

#### Options 
`--network`: Again, this should be the path to your .pkl file.

`--seeds`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)


In [ ]:
!python generate.py --process="truncation" --outdir=./generated/kbhands138-trunc_21 --start=-2.5 --stop=2.5 --increment=0.01 --seeds=21 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl

In [ ]:
!python generate.py --process="truncation" --outdir=./generated/kbhands138-trunc_22 --start=-2.5 --stop=2.5 --increment=0.01 --seeds=22 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl

In [ ]:
!python generate.py --process="truncation" --outdir=./generated/kbhands138-trunc_32 --start=-2.5 --stop=2.5 --increment=0.01 --seeds=32 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl

In [ ]:
!python generate.py --process="truncation" --outdir=./generated/kbhands138-trunc_37 --start=-2.5 --stop=2.5 --increment=0.01 --seeds=37 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl

In [ ]:
!python generate.py --process="truncation" --outdir=./generated/kbhands138-trunc_39 --start=-2.5 --stop=2.5 --increment=0.01 --seeds=39 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl

### Interpolations

Interpolation is the process of generating very small changes to a vector in order to make it appear animated from frame to frame.

We’ll look at different examples of interpolation below.

#### Options

`--network`: path to your .pkl file

`--interpolation`: Walk type defines the type of interpolation you want. In some cases it can also specify whether you want the z space or the w space.

`--frames`: How many frames you want to produce. Use this to manage the length of your video.

`--trunc`: truncation value

#### Linear Interpolation

In [ ]:
!python generate.py --outdir=./generated/kbhands_lerp_z_21-39/ --space="z" --trunc=1 --process="interpolation" --seeds=21,22,32,37,39,21 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl --frames=120 --fps=30

Loading networks from "./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl"...
Generating image for frame 0/600 ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/600 ...
Generating image for frame 2/600 ...
Generating image for frame 3/600 ...
Generating image for frame 4/600 ...
Generating image for frame 5/600 ...
Generating image for frame 6/600 ...
Generating image for frame 7/600 ...
Generating image for frame 8/600 ...
Generating image for frame 9/600 ...
Generating image for frame 10/600 ...
Generating image for frame 11/600 ...
Generating image for frame 12/600 ...
Generating image for frame 13/600 ...
Generating image for frame 14/600 ...
Generating image for frame 15/600 ...
Generating image for frame 16/600 ...
Generating image for frame 17/600 ...
Generating image for frame 18/600 ...
Generating image for frame 19/600 ...
Generating i

In [ ]:
!python generate.py --outdir=./generated/kbhands_lerp_w_21-39/ --space="w" --trunc=1 --process="interpolation" --seeds=21,22,32,37,39,21 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl --frames=120 --fps=30

Loading networks from "./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl"...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Generating image for frame 0/600 ...
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/600 ...
Generating image for frame 2/600 ...
Generating image for frame 3/600 ...
Generating image for frame 4/600 ...
Generating image for frame 5/600 ...
Generating image for frame 6/600 ...
Generating image for frame 7/600 ...
Generating image for frame 8/600 ...
Generating image for frame 9/600 ...
Generating image for frame 10/600 ...
Generating image for frame 11/600 ...
Generating image for frame 12/600 ...
Generating image for frame 13/600 ...
Generating image for frame 14/600 ...
Generating image for frame 15/600 ...
Generating image for frame 16/600 ...
Generating image for frame 17/600 ...
Generating image for frame 18/600 ...
Generating image for frame 19/600 ...
Generating i

In [ ]:
!zip -r vid1.zip /content/out/video1-w-0.5

#### Slerp Interpolation

This gets a little heady, but technically linear interpolations are not the best in high-dimensional GANs. [This github link](https://github.com/soumith/dcgan.torch/issues/14) is one of the more popular explanations ad discussions.

In reality I do not find a huge difference between linear and spherical interpolations (the difference in z- and w-space is enough in many cases), but I’ve implemented slerp here for anyone interested.

Note: Slerp in w space currently isn’t supported. I’m working on it.

In [ ]:
!python generate.py --outdir=./generated/kbhands_slerp_21-39/ --interpolation="slerp" --trunc=1 --process="interpolation" --seeds=21,22,32,37,39,21 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl --frames=120 --fps=30

Loading networks from "./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl"...
Generating image for frame 0/600 ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/600 ...
Generating image for frame 2/600 ...
Generating image for frame 3/600 ...
Generating image for frame 4/600 ...
Generating image for frame 5/600 ...
Generating image for frame 6/600 ...
Generating image for frame 7/600 ...
Generating image for frame 8/600 ...
Generating image for frame 9/600 ...
Generating image for frame 10/600 ...
Generating image for frame 11/600 ...
Generating image for frame 12/600 ...
Generating image for frame 13/600 ...
Generating image for frame 14/600 ...
Generating image for frame 15/600 ...
Generating image for frame 16/600 ...
Generating image for frame 17/600 ...
Generating image for frame 18/600 ...
Generating image for frame 19/600 ...
Generating i

#### Noise Loop

If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path thru the z space to show you a diverse set of images.

`--interpolation="noiseloop"`: set this to use the noise loop funtion

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--random_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

Noise loops currently only work in z space.

In [ ]:
!python generate.py --outdir=./generated/kbhands_noise_loop_21-d01/ --trunc=1 --process="interpolation" --interpolation="noiseloop" --diameter=0.01 --random_seed=21 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl --frames=120 --fps=30

In [ ]:
!python generate.py --outdir=./generated/kbhands_noise_loop_21-d25/ --trunc=1 --process="interpolation" --interpolation="noiseloop" --diameter=0.25 --random_seed=21 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl --frames=120 --fps=30

Loading networks from "./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl"...
Generating image for frame 0/120 ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/120 ...
Generating image for frame 2/120 ...
Generating image for frame 3/120 ...
Generating image for frame 4/120 ...
Generating image for frame 5/120 ...
Generating image for frame 6/120 ...
Generating image for frame 7/120 ...
Generating image for frame 8/120 ...
Generating image for frame 9/120 ...
Generating image for frame 10/120 ...
Generating image for frame 11/120 ...
Generating image for frame 12/120 ...
Generating image for frame 13/120 ...
Generating image for frame 14/120 ...
Generating image for frame 15/120 ...
Generating image for frame 16/120 ...
Generating image for frame 17/120 ...
Generating image for frame 18/120 ...
Generating image for frame 19/120 ...
Generating i

In [ ]:
!python generate.py --outdir=./generated/kbhands_noise_loop_21-d75/ --trunc=1 --process="interpolation" --interpolation="noiseloop" --diameter=0.75 --random_seed=21 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl --frames=120 --fps=30

Loading networks from "./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl"...
Generating image for frame 0/120 ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/120 ...
Generating image for frame 2/120 ...
Generating image for frame 3/120 ...
Generating image for frame 4/120 ...
Generating image for frame 5/120 ...
Generating image for frame 6/120 ...
Generating image for frame 7/120 ...
Generating image for frame 8/120 ...
Generating image for frame 9/120 ...
Generating image for frame 10/120 ...
Generating image for frame 11/120 ...
Generating image for frame 12/120 ...
Generating image for frame 13/120 ...
Generating image for frame 14/120 ...
Generating image for frame 15/120 ...
Generating image for frame 16/120 ...
Generating image for frame 17/120 ...
Generating image for frame 18/120 ...
Generating image for frame 19/120 ...
Generating i

In [ ]:
!python generate.py --outdir=./generated/kbhands_noise_loop_21-d125/ --trunc=1 --process="interpolation" --interpolation="noiseloop" --diameter=1.25 --random_seed=21 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl --frames=120 --fps=30

Loading networks from "./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl"...
Generating image for frame 0/120 ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/120 ...
Generating image for frame 2/120 ...
Generating image for frame 3/120 ...
Generating image for frame 4/120 ...
Generating image for frame 5/120 ...
Generating image for frame 6/120 ...
Generating image for frame 7/120 ...
Generating image for frame 8/120 ...
Generating image for frame 9/120 ...
Generating image for frame 10/120 ...
Generating image for frame 11/120 ...
Generating image for frame 12/120 ...
Generating image for frame 13/120 ...
Generating image for frame 14/120 ...
Generating image for frame 15/120 ...
Generating image for frame 16/120 ...
Generating image for frame 17/120 ...
Generating image for frame 18/120 ...
Generating image for frame 19/120 ...
Generating i

In [ ]:
!python generate.py --outdir=./generated/kbhands_noise_loop_21-d175/ --trunc=1 --process="interpolation" --interpolation="noiseloop" --diameter=1.75 --random_seed=21 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl --frames=120 --fps=30

Loading networks from "./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl"...
Generating image for frame 0/120 ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/120 ...
Generating image for frame 2/120 ...
Generating image for frame 3/120 ...
Generating image for frame 4/120 ...
Generating image for frame 5/120 ...
Generating image for frame 6/120 ...
Generating image for frame 7/120 ...
Generating image for frame 8/120 ...
Generating image for frame 9/120 ...
Generating image for frame 10/120 ...
Generating image for frame 11/120 ...
Generating image for frame 12/120 ...
Generating image for frame 13/120 ...
Generating image for frame 14/120 ...
Generating image for frame 15/120 ...
Generating image for frame 16/120 ...
Generating image for frame 17/120 ...
Generating image for frame 18/120 ...
Generating image for frame 19/120 ...
Generating i

#### Circular Loop

The noise loop is, well, noisy. This circular loop will feel much more even, while still providing a random loop.

I recommend using a higher `--diameter` value than you do with noise loops. Something between `50.0` and `500.0` alongside `--frames` can help control speed and diversity.

In [ ]:
!python generate.py --outdir=./generated/kbhands_in_class-circular_d50/ --trunc=1.25 --process="interpolation" --interpolation="circularloop" --diameter=50.00 --frames=600 --fps=30 --random_seed=1000 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl

Loading networks from "./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl"...
Generating image for frame 0/600 ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/600 ...
Generating image for frame 2/600 ...
Generating image for frame 3/600 ...
Generating image for frame 4/600 ...
Generating image for frame 5/600 ...
Generating image for frame 6/600 ...
Generating image for frame 7/600 ...
Generating image for frame 8/600 ...
Generating image for frame 9/600 ...
Generating image for frame 10/600 ...
Generating image for frame 11/600 ...
Generating image for frame 12/600 ...
Generating image for frame 13/600 ...
Generating image for frame 14/600 ...
Generating image for frame 15/600 ...
Generating image for frame 16/600 ...
Generating image for frame 17/600 ...
Generating image for frame 18/600 ...
Generating image for frame 19/600 ...
Generating i

In [ ]:
!python generate.py --outdir=./generated/kbhands_in_class-circular_d100/ --trunc=1.25 --process="interpolation" --interpolation="circularloop" --diameter=100.00 --frames=600 --fps=30 --random_seed=1000 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl

Loading networks from "./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl"...
Generating image for frame 0/600 ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/600 ...
Generating image for frame 2/600 ...
Generating image for frame 3/600 ...
Generating image for frame 4/600 ...
Generating image for frame 5/600 ...
Generating image for frame 6/600 ...
Generating image for frame 7/600 ...
Generating image for frame 8/600 ...
Generating image for frame 9/600 ...
Generating image for frame 10/600 ...
Generating image for frame 11/600 ...
Generating image for frame 12/600 ...
Generating image for frame 13/600 ...
Generating image for frame 14/600 ...
Generating image for frame 15/600 ...
Generating image for frame 16/600 ...
Generating image for frame 17/600 ...
Generating image for frame 18/600 ...
Generating image for frame 19/600 ...
Generating i

In [ ]:
!python generate.py --outdir=./generated/kbhands_in_class-circular_d200/ --trunc=1.25 --process="interpolation" --interpolation="circularloop" --diameter=200.00 --frames=600 --fps=30 --random_seed=1000 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl

In [ ]:
!python generate.py --outdir=./generated/kbhands_in_class-circular_d400/ --trunc=1.25 --process="interpolation" --interpolation="circularloop" --diameter=400.00 --frames=600 --fps=30 --random_seed=1000 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl

# Projection
Project images into the latent space

In [4]:
!python projector.py --outdir=./generated/projection/output/doug1 --target=./generated/projection/input/doug1.png --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl

Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"...
Computing W midpoint and stddev using 10000 samples...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
step    1/1000: dist 0.60 loss 24567.15
step    2/1000: dist 0.53 loss 27640.43
step    3/1000: dist 0.58 loss 27166.95
step    4/1000: dist 0.55 loss 26253.05
step    5/1000: dist 0.55 loss 24956.84
step    6/1000: dist 0.54 loss 23353.68
step    7/1000: dist 0.53 loss 21512.43
step    8/1000: dist 0.55 loss 19485.85
step    9/1000: dist 0.58 loss 17339.16
step   10/1000: dist 0.46 loss 15139.33
step   11/1000: dist 0.49 loss 12948.90
step   12/1000: dist 0.50 loss 10818.31
step   13/1000: dist 0.55 loss 8801.96
step   14/1000: dist 0.47 loss 6945.81
step   15/1000: dist 0.48 loss 5315.13
step   16/1000: dist 0.46 loss 3970.76
step   17/1000: dist 0.46 loss 2943.90
step   18/1000: dist 0.45 loss 2215.00
step   19/1000: dist 0

## Combine NPZ files together

In [ ]:
!python combine_npz.py --outdir=/content/npz --npzs='/content/pb-proj1-3-clip.npz,/content/pb-proj1-clip-nocenter.npz,/content/pb-proj2-clip-nocenter.npz,/content/projected_w (2).npz'